In [3]:
import sys
freecadpath = "/home/julian/extrepositories/FreeCAD/freecad-build/lib"
sys.path.append(freecadpath)
import FreeCAD
import PathScripts.PathJob as PathJob
import PathScripts.PathLog as PathLog
import PathScripts.PathToolController as PathToolController
import PathScripts.PathUtil as PathUtil


In [4]:
def extractpostlistfromjob(job):
    postlist = []
    currTool = None
    for obj in job.Operations.Group:
        PathLog.debug("obj: {}".format(obj.Name))
        tc = PathUtil.toolControllerForOp(obj)
        if tc is not None:
            if tc.ToolNumber != currTool:
                postlist.append(tc)
                currTool = tc.ToolNumber
        postlist.append(obj)
    return postlist

def extractjobslistfromfile(fname):
    doc = FreeCAD.open(fname)
    jobs = [o  for o in FreeCAD.ActiveDocument.Objects  if hasattr(o, "Proxy") and isinstance(o.Proxy, PathJob.ObjectJob)]
    return jobs

def extractpostlistfromfile(job):
    doc = FreeCAD.open(fname)
    job = doc.getObject("Job")
    return extractpostlistfromjob(job)


In [5]:
fname = "/home/julian/data/freecad/swanneck3.fcstd"
fname = "/home/julian/repositories/transition-CAM/postprocessor/swanneck4.fcstd"
jobs = extractjobslistfromfile(fname)
#postlist = extractpostlistfromfile(fname)
#postlist = extractpostlistfromfile(fname)
print(jobs)

[<Path::FeaturePython object>, <Path::FeaturePython object>, <Path::FeaturePython object>]


In [14]:
jobs[0].Model.Group[0].Shape

In [12]:
postlist = extractpostlistfromjob(jobs[1])
print(postlist)

[<Path::FeaturePython object>, <Path::FeaturePython object>]


In [13]:
from kineticnc_post import flattenandgroup
tooldefmotions = flattenandgroup(postlist)


{'lastpos': {'Y': 233.0, 'X': 60.387275, 'Z': 30.0, 'F': 750.0}, 'S': 10000.0, 'cycletype': 'normal', 'firstpos': {'Y': 233.0, 'X': 60.387275, 'Z': 30.0, 'F': 100.0}, 'T': 2.0}


In [14]:
tooldefmotions[0][0]

{'S': 10000.0,
 'T': 2.0,
 'cycletype': 'normal',
 'firstpos': {'F': 100.0, 'X': 60.387275, 'Y': 233.0, 'Z': 30.0},
 'lastpos': {'F': 750.0, 'X': 60.387275, 'Y': 233.0, 'Z': 30.0}}

In [15]:
from StringIO import StringIO
from kineticnc_post import writetooldefheader, writedrillmotioncmds, writemotioncmds, writefilefooter

fout = StringIO()
currpos = {}
for i, (tooldef, motioncmds) in enumerate(tooldefmotions):
    tooldef["filename"] = "None"
    writetooldefheader(fout, tooldef, i, currpos)
    if tooldef["cycletype"] == "drill": 
        writedrillmotioncmds(fout, motioncmds, currpos)
    else:
        writemotioncmds(fout, motioncmds, currpos)
writefilefooter(fout, currpos)


In [16]:
print(fout.getvalue())

%
(Start None)
(Exported by FreeCAD)
(Post Processor: kineticnc_post)
(Output Time: 2018-12-31 18:16:13.046557)
G90 (Absolute coordinates)
G17 (XY Plane selection)
G21 (Programming in mm)
G94 (Feedrate per minute)

(Diameter: unknown)
M6 T2 (Change tool)
M3 S10000 (Spindle on)
G0 Z30.000 (To clearance plane)
G0 X60.387 Y233.000
G1 Z30.000 F100 (Set feedrate on spot)
G0 
Z25.000 
G1 Z11.000 
G2 X57.287 Z10.900 I-1.550 J0.000 F750.000 
G2 X60.387 Z10.800 I1.550 J0.000 
G2 X57.287 Z10.700 I-1.550 J0.000 
G2 X60.387 Z10.600 I1.550 J0.000 
G2 X57.287 Z10.500 I-1.550 J0.000 
G2 X60.387 Z10.400 I1.550 J0.000 
G2 X57.287 Z10.300 I-1.550 J0.000 
G2 X60.387 Z10.200 I1.550 J0.000 
G2 X57.287 Z10.100 I-1.550 J0.000 
G2 X60.387 Z10.000 I1.550 J0.000 
G2 X57.287 Z9.900 I-1.550 J0.000 
G2 X60.387 Z9.800 I1.550 J0.000 
G2 X57.287 Z9.700 I-1.550 J0.000 
G2 X60.387 Z9.600 I1.550 J0.000 
G2 X57.287 Z9.500 I-1.550 J0.000 
G2 X60.387 Z9.400 I1.550 J0.000 
G2 X57.287 Z9.300 I-1.550 J0.000 
G2 X60.387 Z9.200